In [1]:
import pandas as pd
import yfinance as yf
import pyarrow.parquet as pq
from datetime import datetime, timedelta
import os

In [6]:
list_tickers= ["BTC-USD", "ETH-USD", "ADA-USD", "SPY"]
folder= "data"
file_name= "historical_data.parquet"
file_path= os.path.join(folder, file_name)


In [7]:
initial_date= "2020-01-01"

yesterday_date = datetime.today() - timedelta(days=1)
last_week_date = yesterday_date - timedelta(days=7)

### Get Data

In [9]:
def get_historical_data(list_tickers, start_date, end_date):
    try:
        # Initialize an empty DataFrame to store the data
        data = pd.DataFrame()
        
        # Get data for each stock in list tickers
        for symbol in list_tickers:
            stock_data = yf.download(symbol, start=start_date, end=end_date)
            stock_data['Symbol'] = symbol  # Add a column for the stock symbol
            stock_data['Year'] = stock_data.index.year
            data = pd.concat([data, stock_data], axis=0)  # Concatenate data

        data.reset_index(inplace=True)   
            
        return data
    
    except Exception as e:
        print("Error fetching data:", e)
        return None

def store_data(df, folder_path, prefix):
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Get current date for partitioning
    current_date = datetime.today().strftime('%Y-%m-%d')
    
    # Construct file path with prefix and current date
    file_name = f"{prefix}.parquet"
    folder= "data"
    file_path = os.path.join(folder, file_name)
    
    # Write DataFrame to partitioned Parquet file
    df.to_parquet(file_path)
    
    print(f"Data has been successfully written to {file_path}")

if __name__ == "__main__":
    # Check if historical_data.parquet file exists
    if os.path.exists(file_path):
        print("Historical data file exists. Fetching last seven days' data.")
        historical_data = get_historical_data(list_tickers, start_date=last_week_date, end_date=yesterday_date)
        store_data(historical_data, data_folder, prefix=f"crypto")
    else:
        print("Historical data file does not exist. Fetching historical data.")
        historical_data = get_historical_data(list_tickers, start_date=initial_date, end_date=yesterday_date)
        store_data(historical_data, data_folder, prefix="historical_data")

Historical data file exists. Fetching last seven days' data.


c:\Users\T-Gamer\Documents\3.dgrubert\cripto-forecast\.venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Users\T-Gamer\Documents\3.dgrubert\cripto-forecast\.venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Users\T-Gamer\Documents\3.dgrubert\cripto-forecast\.venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  

Data has been successfully written to data\crypto.parquet


In [36]:
#test
pd.read_parquet("data/historical_data.parquet")

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Year
0,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997,BTC-USD,2020
1,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465,BTC-USD,2020
2,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032,BTC-USD,2020
3,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275,BTC-USD,2020
4,2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095,BTC-USD,2020
...,...,...,...,...,...,...,...,...,...
5561,2024-02-12,501.170013,503.500000,500.239990,500.980011,500.980011,56502300,SPY,2024
5562,2024-02-13,494.529999,497.089996,490.720001,494.079987,494.079987,113099200,SPY,2024
5563,2024-02-14,496.790009,499.070007,494.399994,498.570007,498.570007,68387800,SPY,2024
5564,2024-02-15,499.290009,502.200012,498.799988,502.010010,502.010010,61683000,SPY,2024
